In [1]:
%pip install --quiet transformers==4.37.2 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.41.2.post2 datasets==2.14.7

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2024.1.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2024.1.0 which is incompatible.
dask-cuda 23.8.0 require

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm, trange
import torch
import torch.nn as nn
import torch.nn.functional as F
import peft

import transformers
from datasets import load_dataset

import random
const_seed = 100

In [3]:
torch.cuda.is_available()

True

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Part 0: Initializing the model and tokenizer

let's take mistral model for our experiments (https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2) that was tuned to follow user instructions. Pay attention that we load model in 4 bit to decrease the memory usage.

In [5]:
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'


# load llama tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

# Note: to speed up inference you can use flash attention 2 (https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-2)
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
for param in model.parameters():
    param.requires_grad=False

model.gradient_checkpointing_enable()  # only store a small subset of activations, re-compute the rest.
model.enable_input_require_grads()     # override an implementation quirk in gradient checkpoints that disables backprop unless inputs require grad
# more on gradient checkpointing: https://pytorch.org/docs/stable/checkpoint.html https://arxiv.org/abs/1604.06174

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

# Part 1 (5 points): Prompt-engineering

**There are different strategies for text generation in huggingface:**

| Strategy | Description | Pros & Cons |
| --- | --- | --- |
| Greedy Search | Chooses the word with the highest probability as the next word in the sequence. | **Pros:** Simple and fast. <br> **Cons:** Can lead to repetitive and incoherent text. |
| Sampling with Temperature | Introduces randomness in the word selection. A higher temperature leads to more randomness. | **Pros:** Allows exploration and diverse output. <br> **Cons:** Higher temperatures can lead to nonsensical outputs. |
| Nucleus Sampling (Top-p Sampling) | Selects the next word from a truncated vocabulary, the "nucleus" of words that have a cumulative probability exceeding a pre-specified threshold (p). | **Pros:** Balances diversity and quality. <br> **Cons:** Setting an optimal 'p' can be tricky. |
| Beam Search | Explores multiple hypotheses (sequences of words) at each step, and keeps the 'k' most likely, where 'k' is the beam width. | **Pros:** Produces more reliable results than greedy search. <br> **Cons:** Can lack diversity and lead to generic responses. |
| Top-k Sampling | Randomly selects the next word from the top 'k' words with the highest probabilities. | **Pros:** Introduces randomness, increasing output diversity. <br> **Cons:** Random selection can sometimes lead to less coherent outputs. |
| Length Normalization | Prevents the model from favoring shorter sequences by dividing the log probabilities by the sequence length raised to some power. | **Pros:** Makes longer and potentially more informative sequences more likely. <br> **Cons:** Tuning the normalization factor can be difficult. |
| Stochastic Beam Search | Introduces randomness into the selection process of the 'k' hypotheses in beam search. | **Pros:** Increases diversity in the generated text. <br> **Cons:** The trade-off between diversity and quality can be tricky to manage. |
| Decoding with Minimum Bayes Risk (MBR) | Chooses the hypothesis (out of many) that minimizes expected loss under a loss function. | **Pros:** Optimizes the output according to a specific loss function. <br> **Cons:** Computationally more complex and requires a good loss function. |

Documentation references:
- [reference for `AutoModelForCausalLM.generate()`](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationMixin.generate)
- [reference for `AutoTokenizer.decode()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)
- Huggingface [docs on generation strategies](https://huggingface.co/docs/transformers/generation_strategies)

In [20]:
# TODO: create a function for generation with huggingface

def get_answer(tokenizer, model, messages, max_new_tokens=500,
               temperature=1, do_sample=True):
    input_message = " ".join(message["content"] for message in messages)

    inputs = tokenizer.encode(input_message, return_tensors="pt")

    inputs = inputs.to(model.device)

    outputs = model.generate(
        inputs,
        max_length=len(inputs[0]) + max_new_tokens,
        temperature=temperature,
        do_sample=do_sample,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=torch.ones_like(inputs)
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return decoded


In [104]:
def get_answer(tokenizer, model, messages, max_new_tokens=200, 
               temperature=0.5, do_sample=True):
    
    user_messages = " ".join(message["content"] for message in messages if message["role"] == "user")

    inputs = tokenizer.encode(user_messages, return_tensors="pt")

    with tokenizer.as_target_tokenizer():
        outputs = model.generate(inputs, max_length=len(inputs[0])+max_new_tokens, 
                                 do_sample=do_sample, temperature=temperature, pad_token_id=tokenizer.eos_token_id)

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return decoded


In [105]:
# Let's try our model

messages = [
    {"role": "user", "content": "Write an explanation of tensors for 5 year old"},
]

print(get_answer(tokenizer, model, messages))

Write an explanation of tensors for 5 year olds.

Tensors are like special boxes that hold different things. But instead of holding just one thing like a regular box, a tensor box can hold many things at once. And not just any things, but things that have shapes and sizes. For example, a box can hold balls of different sizes, or blocks of different shapes.

When we play with these tensor boxes, we can do interesting things with them. We can add or take away things from one box and put them into another box. And because these tensor boxes can hold many things at once, we can do these operations in many directions. We can add or take away things not just lengthwise, but also widthwise and heightwise.

So, tensors are like special boxes that can hold many things of different shapes and sizes, and we can do operations on them in many directions.


You should obtain an explanation from the model. If so, let us go further!

Now we will take a sample from boolQ (https://huggingface.co/datasets/google/boolq) dataset and try prompting techniques to extract the needed answer and calculate its quality

In [8]:
df = load_dataset("google/boolq")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9427 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3270 [00:00<?, ? examples/s]

In [107]:
# Fixing 20 validation examples

random.seed(const_seed)
idx = random.sample(range(1, 3270), 20)

In [108]:
# sample you will work with
df_sample = df["validation"].select(idx)

KeyError: 'validation'

In [109]:
df_sample

Dataset({
    features: ['question', 'answer', 'passage'],
    num_rows: 20
})

In [110]:
# For instance, you can construct your prompt the following way
messages = [
    {"role": "user", "content": '''You are given a text and question. Answer only "true" or "false".
text: As with other games in The Elder Scrolls series, the game is set on the continent of Tamriel. The events of the game occur a millennium before those of The Elder Scrolls V: Skyrim and around 800 years before The Elder Scrolls III: Morrowind and The Elder Scrolls IV: Oblivion. It has a broadly similar structure to Skyrim, with two separate conflicts progressing at the same time, one with the fate of the world in the balance, and one where the prize is supreme power on Tamriel. In The Elder Scrolls Online, the first struggle is against the Daedric Prince Molag Bal, who is attempting to meld the plane of Mundus with his realm of Coldharbour, and the second is to capture the vacant imperial throne, contested by three alliances of the mortal races. The player character has been sacrificed to Molag Bal, and Molag Bal has stolen their soul, the recovery of which is the primary game objective.
question: is elder scrolls online the same as skyrim
answer: '''},
]

print(get_answer(tokenizer, model, messages)[0])

Y


In [111]:
messages = [
    {"role": "user", "content": '''You are given a text and question. Answer only "true" or "false".
text: As with other games in The Elder Scrolls series, the game is set on the continent of Tamriel. The events of the game occur a millennium before those of The Elder Scrolls V: Skyrim and around 800 years before The Elder Scrolls III: Morrowind and The Elder Scrolls IV: Oblivion. It has a broadly similar structure to Skyrim, with two separate conflicts progressing at the same time, one with the fate of the world in the balance, and one where the prize is supreme power on Tamriel. In The Elder Scrolls Online, the first struggle is against the Daedric Prince Molag Bal, who is attempting to meld the plane of Mundus with his realm of Coldharbour, and the second is to capture the vacant imperial throne, contested by three alliances of the mortal races. The player character has been sacrificed to Molag Bal, and Molag Bal has stolen their soul, the recovery of which is the primary game objective.
question: is elder scrolls online the same as skyrim
answer: '''},
]

print(get_answer(tokenizer, model, messages))

You are given a text and question. Answer only "true" or "false".
text: As with other games in The Elder Scrolls series, the game is set on the continent of Tamriel. The events of the game occur a millennium before those of The Elder Scrolls V: Skyrim and around 800 years before The Elder Scrolls III: Morrowind and The Elder Scrolls IV: Oblivion. It has a broadly similar structure to Skyrim, with two separate conflicts progressing at the same time, one with the fate of the world in the balance, and one where the prize is supreme power on Tamriel. In The Elder Scrolls Online, the first struggle is against the Daedric Prince Molag Bal, who is attempting to meld the plane of Mundus with his realm of Coldharbour, and the second is to capture the vacant imperial throne, contested by three alliances of the mortal races. The player character has been sacrificed to Molag Bal, and Molag Bal has stolen their soul, the recovery of which is the primary game objective.
question: is elder scrolls on

In [121]:
messages = []

for i in range(len(df_sample)):
    passage = df_sample['passage'][i]
    question = df_sample['question'][i]

    message = {
        "role": "user",
        "content": f'''Answer only "true" or "false" according to the question and text. Write only True or False and that's all!
text: {passage}
question: {question}
answer: '''
    }

    messages.append(message)


In [147]:
generated_answer_new = []
for i, message in enumerate(messages):
    prompt = [message]
    output = get_answer(tokenizer, model, prompt)
    generated_answer_new.append(output)
    print(f"Answer for message {i+1}: {output}")

Answer for message 1: Answer only "true" or "false" according to the question and text. Write only True or False and that's all!
text: As the Senate president, the vice president presides over its deliberations (or delegates this task to a member of the Senate), but is allowed to vote only when it is necessary to break a tie. While this vote-casting prerogative has been exercised chiefly on legislative issues, it has also been used to break ties on the election of Senate officers, as well as on the appointment of Senate committees. In this capacity, the vice president also presides over joint sessions of Congress.
question: is the vice president the head of the senate
answer: 
True. The vice president presides over the Senate and is considered the president of the Senate. However, they only have the power to vote when necessary to break a tie.
Answer for message 2: Answer only "true" or "false" according to the question and text. Write only True or False and that's all!
text: The Feder

In [155]:
df_sample['new_answer']= generated_answer_new
df_sample['new_answer'] = df_sample['new_answer'].apply(lambda x: '\n'.join(x.split('\n')[1:]))
df_sample['new_answer']

0     text: As the Senate president, the vice presid...
1     text: The Federal Reserve began taking high-de...
2     text: On the outbreak of war, the Confederates...
3     text: The ground squirrels are members of the ...
4     text: Kim Garner, the senior vice president of...
5     text: Compared to similar technology in other ...
6     text: The Boss Baby: Back in Business is an Am...
7     text: Baby back ribs (also back ribs or loin r...
8     text: The climate in the region is generally c...
9     text: The away goals rule is applied in many f...
10    text: Nigella sativa (black caraway, also know...
11    text: Belgium have appeared in the finals tour...
12    text: In 2003, the United States withdrew rema...
13    text: Each legislator shall be at least twenty...
14    text: Brie (/briː/; French: (bʁi)) is a soft c...
15    text: In Australia, each state has its own con...
16    text: A table may have multiple foreign keys, ...
17    text: Delay of game is a penalty in ice ho

Is anything wrong with the output? Now it is time for you to play around and try to come up with some better prompt.

In [157]:
# TODO: create function to evaluate answers
# Note: you can adapt function for different answer structures,
# but you should be able to automatically extract the target "true" or "false" components

def extract_true_false_from_answer_column(answer_column):
    true_false_list = []
    for answer in answer_column:
        lines = answer.split("\n")
        found = False
        for line in lines:
            if "true" in line.lower():
                true_false_list.append("true")
                found = True
                break
            elif "false" in line.lower():
                true_false_list.append("false")
                found = True
                break
        if not found:
            true_false_list.append("unknown")
    return true_false_list

true_false_list = extract_true_false_from_answer_column(df_sample['new_answer'])
df_sample['generated_answer'] = true_false_list
print(true_false_list)


['true', 'false', 'true', 'false', 'false', 'true', 'true', 'false', 'true', 'true', 'true', 'false', 'true', 'true', 'true', 'true', 'true', 'true', 'false', 'false']


In [164]:
def evaluate_answers(generated_answers, expected_true_false):
    correct_count = 0
    total_count = len(expected_true_false)

    for generated_answer, expected_answer in zip(generated_answers, expected_true_false):
        if generated_answer == expected_answer:
            correct_count += 1

    accuracy = correct_count / total_count if total_count > 0 else 0
    return accuracy

expected_true_false = df_sample['answer'].astype(str).str.lower().tolist()

generated_true_false = df_sample['generated_answer'].tolist()

accuracy = evaluate_answers(generated_true_false, expected_true_false)
print("Accuracy:", accuracy)


Accuracy: 0.75


In [166]:
df_sample.to_csv('new_dataset.csv')

TODO: Try and compare "naive" prompting (your best hand-crafted variant), few-shot prompting (https://www.promptingguide.ai/techniques/fewshot) and chain-of-thought prompting (step-be-step thinking - https://www.promptingguide.ai/techniques/cot).

Save the generation results into separate csv files and do not forget to attach them to your homework.

In [172]:
messages_naive = []

for i in range(len(df_sample)):
    passage = df_sample['passage'][i]
    question = df_sample['question'][i]

    message = {
        "role": "user",
        "content": f'''Answer only "true" or "false" according to the question and text. Write only True or False and that's all!
text: {passage}
question: {question}
answer: '''
    }

    messages_naive.append(message)

    
messages_few_shot = []

for i in range(len(df_sample)):
    passage = df_sample['passage'][i]
    question = df_sample['question'][i]

    message = {
        "role": "user",
        "content": f'''Provide your answer as "true" or "false" based on the few-shot examples given below.
text: {passage}
question: {question}
Few-shot examples:
- True: [Provide a true statement example related to the question]
- False: [Provide a false statement example related to the question]
answer: '''
    }

    messages_few_shot.append(message)

messages_chain_of_thought = []

for i in range(len(df_sample)):
    passage = df_sample['passage'][i]
    question = df_sample['question'][i]

    message = {
        "role": "user",
        "content": f'''Consider the following steps and respond accordingly:
Step 1: Read the provided text.
Step 2: Consider the question asked.
Step 3: Based on your understanding from Step 1 and Step 2, determine if the statement is true or false.
text: {passage}
question: {question}
answer: '''
    }

    messages_chain_of_thought.append(message)


In [173]:
generated_responses_naive = []
generated_responses_few_shot = []
generated_responses_chain_of_thought = []

for i, message in enumerate(messages_naive):
    prompt = [message]
    output = get_answer(tokenizer, model, prompt)
    generated_responses_naive.append(output)

for i, message in enumerate(messages_few_shot):
    prompt = [message]
    output = get_answer(tokenizer, model, prompt)
    generated_responses_few_shot.append(output)

for i, message in enumerate(messages_chain_of_thought):
    prompt = [message]
    output = get_answer(tokenizer, model, prompt)
    generated_responses_chain_of_thought.append(output)


In [174]:
import csv
def save_to_csv(filename, generated_responses):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Message', 'Generated Response'])
        for i, response in enumerate(generated_responses):
            writer.writerow([f"Message {i+1}", response])

save_to_csv('generated_responses_naive.csv', generated_responses_naive)
save_to_csv('generated_responses_few_shot.csv', generated_responses_few_shot)
save_to_csv('generated_responses_chain_of_thought.csv', generated_responses_chain_of_thought)


# Part 2 (5 points): Fine-tuning with PEFT and LoRA

In [176]:
peft_config = peft.PromptTuningConfig(task_type=peft.TaskType.CAUSAL_LM,
                                      num_virtual_tokens=16) #
model = peft.get_peft_model(model, peft_config)  # note: for most peft methods, this line also modifies model in-plac)))

In [ ]:
model.print_trainable_parameters() # Wow so small amount of trainable params

In [ ]:
# creating simple prompt formating
def format_prompt(sample):
    return f'''
    text: {sample['passage']}
    question: {sample['question']}
    answer: {sample['answer']}
    '''

TODO: initialize Trainer and pass train part of our dataset for 2-3 epoches

Note: carefully set max_seq_length and args (that are transformers.TrainingArguments)

TODO: save and check your tuned model. Provide scores on our 20 validation examples and save result to csv file